In [1]:
from sklearn.datasets import load_diabetes
import statsmodels.api as sm
import numpy as np
from pandas import DataFrame
data = load_diabetes()
Xa = data['data']
ya = data['target']
features = data['feature_names']
targets = ['target']
diabetes = DataFrame(np.c_[Xa, ya], columns = features+targets)

# 단순 선형 회귀 적합을 위해 전체 데이터에서 4개의 설명변수와 1개의 종속변수를 추출한다.
X = diabetes.filter(['bmi', 'age', 'sex', 'bp'])
y = diabetes.filter(['target'])

# 회귀 분석 fitted객체, 요약결과 함수
def multiR_fitted(X, y): #return fitted, summary
    X_ = sm.add_constant(X)
    model = sm.OLS(y, X_)
    fitted = model.fit()
    return fitted, fitted.summary()

fitted, summary = multiR_fitted(X, y)
summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.400
Model:                            OLS   Adj. R-squared:                  0.395
Method:                 Least Squares   F-statistic:                     72.91
Date:                Sat, 20 May 2023   Prob (F-statistic):           2.70e-47
Time:                        00:12:30   Log-Likelihood:                -2434.2
No. Observations:                 442   AIC:                             4878.
Df Residuals:                     437   BIC:                             4899.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        152.1335      2.853     53.329      0.000     146.527     157.740
bmi          787.1817     65.424     12.032      0.000     658.597     915.766
age           37.2406     64.117      0.581      0.562     -88.776     163.258
sex         -106.5762     62.125     -1.716      0.087    -228.677      15.525
bp           416.6725     69.495      5.996      0.000     280.087     553.258
==============================================================================
Omnibus:                        9.858   Durbin-Watson:                   1.933
Prob(Omnibus):                  0.007   Jarque-Bera (JB):                6.464
Skew:                           0.146   Prob(JB):                       0.0395
Kurtosis:                       2.485   Cond. No.                         28.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [2]:
# 모든 변수를 포함한 full model
print("full model = const, bp, bmi, age, sex")
R2 = fitted.rsquared
n = fitted.nobs #number of observations
p = fitted.df_model #설명변수의 개수
constant = 'const' in fitted.params.keys() #절편 유무
SSE = sum((fitted.resid)**2) #full model의 잔차제곱합
MSE = SSE / (n-p-1) # fitted.mse_resid (MSE)

adj_R2 = 1- (1-R2)*(n-1)/(n-p-1)
print(" 수정 결정계수: {:.3f}".format(adj_R2))

llf = -n/2*np.log(2*np.pi) - n/2*np.log(SSE / n) - n/2 #로그 가능도 값
aic = -2*llf + 2*(p + constant)
bic = -2*llf + np.log(n)*(p + constant)
print(" AIC: {:.1f}".format(aic))
print(" BIC: {:.1f}".format(bic))

cp = SSE/MSE - n + 2*(p+1)
print(" Cp {:.1f} -> 변수+상수 개수의 합 {}".format(cp, p+constant))

full model = const, bp, bmi, age, sex
 수정 결정계수: 0.395
 AIC: 4878.4
 BIC: 4898.8
 Cp 5.0 -> 변수+상수 개수의 합 5.0


In [3]:
# 일부 변수만 포함한 후보 model
print("후보 model = const, bp, bmi")
fitted2, summary2 = multiR_fitted(X.filter(['bp', 'bmi']), y)
n = fitted2.nobs #number of observations
p = fitted2.df_model #설명변수의 개수
constant = 'const' in fitted2.params.keys() #절편 유무
SSE = sum((fitted2.resid)**2) #full model의 잔차제곱합
MSE = fitted.mse_resid # 비교모델의 MSE가 아닌 full모델의 MSE

adj_R2 = 1- (1-R2)*(n-1)/(n-p-1)
print(" 수정 결정계수: {:.3f}".format(adj_R2))

llf = -n/2*np.log(2*np.pi) - n/2*np.log(SSE / n) - n/2 #로그 가능도 값
aic = -2*llf + 2*(p + constant)
bic = -2*llf + np.log(n)*(p + constant)
print(" AIC: {:.1f}".format(aic))
print(" BIC: {:.1f}".format(bic))

cp = SSE/MSE - n + 2*(p+1)
print(" Cp {:.1f} -> 변수+상수 개수의 합 {}".format(cp, p+constant))

후보 model = const, bp, bmi
 수정 결정계수: 0.398
 AIC: 4877.5
 BIC: 4889.8
 Cp 4.1 -> 변수+상수 개수의 합 3.0


In [4]:
## 단계적 선택법
from pandas import DataFrame
import statsmodels.api as sm
from itertools import combinations
import numpy as np
def stepwise_method(X, y, criterion='AIC'): #'BIC', 'CP', 'adj_R2'
    result = DataFrame()
    feature_combis = [] #변수 조합의 모든 경우의 수
    for i in range(1, len(X.columns)+1):
        feature_combis += list(combinations(X.columns, i))
    feature_combis.reverse() #p개수 내림차순
    
    for j, feature_combi in enumerate(feature_combis):
        X_ = X.filter(feature_combi)
        X_ = sm.add_constant(X_) #절편 추가
        model = sm.OLS(y, X_)
        fitted = model.fit() #모델 적합
        n = fitted.nobs #number of observations
        p = fitted.df_model #설명변수의 개수
        if j==0:
            MSE = fitted.mse_resid # full모델의 MSE
        
        #각 기준값 계산
        aic = fitted.aic
        bic = fitted.bic
        cp = sum((fitted.resid)**2)/MSE - (n-2*(p+1))
        adj_R2 = fitted.rsquared_adj
        
        #각 기준값 입력
        result.loc[j, 'feature_combi'] = ", ".join(list(fitted.params.keys()))
        result.loc[j, 'AIC'] = aic
        result.loc[j, 'BIC'] = bic
        result.loc[j, 'CP'] = cp
        result.loc[j, 'adj_R2'] = adj_R2
        
        if criterion in ['AIC', 'BIC']: #낮을수록 Best
            result = result.sort_values(by=criterion, ascending = True)
            best = result.iloc[0, 0]
        elif criterion in ['adj_R2']: #높을수록 Best
            result = result.sort_values(by=criterion, ascending = False)
            best = result.iloc[0, 0]
        elif criterion in ['CP']: #CP값이 작고, 변수의 개수와 유사할수록 Best
            best_idx = np.abs(result['CP']-result['feature_combi'].apply(lambda x: x.count(',')+1)).sort_values(ascending=True).index
            result = result.loc[best_idx, :]
            best = result.iloc[0, 0]
        else:
            print("criterion options only cover AIC, BIC, CP, adj_R2.")
    return best, result

In [5]:
# AIC 기준으로 단계적 선택법을 통한 최적의 변수 선택 결과는 다음과 같다.
best, result = stepwise_method(X, y, criterion='CP')
print("최적의 변수 조합: ", best)
print(f"전체 결과값: \n{result}\n")

for c in ['AIC', 'BIC', 'CP', 'adj_R2']:
    best, result = stepwise_method(X, y, c)
    print(f"기준 {c}에 의한 최적의 변수 조합 {best}")

최적의 변수 조합:  const, bmi, age, sex, bp
전체 결과값: 
               feature_combi          AIC          BIC          CP    adj_R2
0   const, bmi, age, sex, bp  4878.354373  4898.810922    5.000000  0.394771
2        const, bmi, sex, bp  4876.695451  4893.060691    3.337350  0.395687
8             const, bmi, bp  4877.487843  4889.761772    4.109030  0.393242
3        const, bmi, age, bp  4879.321047  4895.686287    5.942980  0.392087
10           const, bmi, age  4909.624337  4921.898267   37.300027  0.347484
4       const, bmi, age, sex  4911.296550  4927.661789   38.949157  0.346479
14                const, bmi  4912.038221  4920.220840   40.050156  0.342433
9            const, bmi, sex  4913.987529  4926.261459   41.995333  0.341010
1        const, age, sex, bp  5002.828421  5019.193660  147.769066  0.196111
5             const, sex, bp  5002.163006  5014.436935  147.528327  0.195517
6             const, age, bp  5003.526444  5015.800374  149.331118  0.193032
11                 const, bp  